In [1]:
import pandas as pd


In [10]:
import os
data_dir = '/Users/bijubiju/Desktop/databases/australia_property_data/nsw/2023'
zip_dat_files = os.listdir(data_dir)
zip_dat_files = [f for f in zip_dat_files if f.endswith('.zip')]


In [29]:
import zipfile
def parse_title_references(zip_path,dat_filename):
    """
    Parse title references from a DAT file
    
    Args:
        file_path (str): Path to the DAT file
    """
    # Read the file
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        with zip_file.open(dat_filename) as file:
            # Read and decode the file
            data = file.read().decode('utf-8')
    # Split the data into lines and filter for C records
    lines = [line.strip() for line in data.split('\n') if line.strip().startswith('B')]
    
    # Parse each C record
    records = []
    for line in lines:
        fields = line.split(';')
        record = {
            'district_code': fields[1],
            'property_id': fields[2],
            'sale_counter': fields[3],
            'download_datetime': fields[4],
            'property_name': fields[5],
            'unit_number': fields[6],
            'house_number': fields[7],
            'street_name': fields[8],
            'locality': fields[9],
            'post_code': fields[10],
            'area': fields[11],
            'area_type': fields[12],
            'contract_date': fields[13],
            'settlement_date': fields[14],
            'purchase_price': fields[15],
            'zoning': fields[16],
            'nature_of_property': fields[17],
            'primary_purpose': fields[18],
            'strata_lot_number': fields[19],
            'component_code': fields[20],
            'sale_code': fields[21],
            'interest_percentage': fields[22],
            'dealing_number': fields[23]
        }
        records.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(records)
    
    # Convert dates and numeric fields
    df['download_datetime'] = pd.to_datetime(df['download_datetime'], format='%Y%m%d %H:%M', errors='coerce')
    df['contract_date'] = pd.to_datetime(df['contract_date'], format='%Y%m%d', errors='coerce')
    df['settlement_date'] = pd.to_datetime(df['settlement_date'], format='%Y%m%d', errors='coerce')
    df['purchase_price'] = pd.to_numeric(df['purchase_price'], errors='coerce')
    df['area'] = pd.to_numeric(df['area'], errors='coerce')
    
    return df

# Example usage


In [36]:
from zipfile import ZipFile
all_dfs = []
for zip_file in zip_dat_files:
        zip_path = os.path.join(data_dir, zip_file)
        
        with ZipFile(zip_path, 'r') as zip_ref:
        
            # Get list of .dat files in the ZIP
            
            dat_files = [f for f in zip_ref.namelist() if f.endswith('.DAT')]
            
            # Process each .dat file
            for dat_file in dat_files:
                
                # Read directly from the ZIP file without extracting
                #zip_path = "/Users/bijubiju/Desktop/databases/australia_property_data/nsw/2023/20230102.zip"
                #dat_filename = "004_SALES_DATA_NNME_02012023.DAT"

                df =parse_title_references(zip_path=zip_path,dat_filename=dat_file)
                all_dfs.append(df)
    
full_df = pd.concat(all_dfs,ignore_index=True)

In [37]:
full_df.to_csv('2023_property_data')